# Preprocessing

## Importo le librerie utili

In [1]:
#La keyword import mi permette di utilizzare le funzioni presenti in altre librerie all'interno del mio script
#NB: Le librerie prima vanno scaricate

import numpy as np
import os
import pandas as pd
import mne

from mne.preprocessing import ICA

#Per utilizzare una funzione da queste librerie devo scrivere: nome_libreria.nome_funzione()

***

## Importo l'EEG del paziente che voglio studiare

Creo una stringa di testo contenente l'indirizzo della cartella dove sono gli eeg

In [2]:
#Se avete messo il dataset nella stessa cartella del notebook potete usare questa riga, crea la stringa contenente il percorso
#della working directory
#home_path = os.path.abspath(os.getcwd())

#altrimenti usate questa
home_path="E:\\Datasets\\NL3"

#la vostra sarà diversa, dovrete cambiarla (Tasto dx del mouse sulla cartella + 'copia come percorso')
#Python legge '\' dei percorsi come un carattere speciale quindi quando li scrivete dovete sostituirlo con '\\'

print(home_path)

E:\Datasets\NL3


Scelgo l'indice del paziente che voglio studiare

In [3]:
paziente = 5

Creo la stringa contenente l'indirizo del file dell'EEG del paziente che ho scelto
NB: str() converte un numero in una stringa e per concatenare più stringhe in python si può usare il +

In [4]:
if paziente<10:
        filename = home_path+"\eeg\sub-0"+str(paziente)+"\eeg\sub-0"+str(paziente)+"_task-rsvp_eeg.vhdr"
if paziente >= 10:
        filename = home_path+"\eeg\sub-"+str(paziente)+"\eeg\sub-"+str(paziente)+"_task-rsvp_eeg.vhdr"
        
print(filename)

E:\Datasets\NL3\eeg\sub-05\eeg\sub-05_task-rsvp_eeg.vhdr


Utilizzo la funzione di mne "read_raw_brainvision" per importare l'eeg del paziente

In [5]:
raw=mne.io.read_raw_brainvision(filename, preload=True)

Extracting parameters from E:\Datasets\NL3\eeg\sub-05\eeg\sub-05_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3022019  =      0.000 ...  3022.019 secs...


Eseguo il plot dell'EEG

In [ ]:
#Plot di 3 secondi di 10 canali dell'EEG
raw.plot(duration = 3, n_channels = 10)

Using matplotlib as 2D backend.


Eseguo il plot della Power Spectral Density

In [ ]:
#Plot della Power Spectral Density dei canali
raw.compute_psd().plot()

#### Provate a vedere pazienti e canali diversi!

***

## Filtri e Resampling

In [ ]:
#Applico un filtro FIR tra 0.1 e 12 Hz per isolare le frequenze rilevanti
filtro=raw.filter(0.1, 12, method='fir')

In [9]:
reference=filtro.set_eeg_reference(ref_channels='average')

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [10]:
#Il dataset ha un sample ratio di 1000 Hz, che è troppo alto, quindi faccio un resampling e lo porto a 250 Hz
resample=reference.copy().resample(sfreq=250)
notch=resample.copy().notch_filter(freqs=10)

Setting up band-stop filter from 9.5 - 11 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 9.47
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 9.22 Hz)
- Upper passband edge: 10.53 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 10.78 Hz)
- Filter length: 1651 samples (6.604 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    1.5s finished


## ICA - Independent Component Analysis

In [11]:
#Per utilizzare l'ICA è consigliato applicare un filtro passa-alto di 1 Hz
preica=notch.copy().filter(l_freq=1., h_freq=None)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    1.6s finished


In [12]:
#Imposto la bipolar reference
#Uso i canali prefrontali per costruire un nuovo canale artificiale che sostituirà l'EOG nella ricerca di artefatti oculari

bipolar_ref=mne.set_bipolar_reference(preica, 'Fp1', 'Fp2', ch_name='Reference', drop_refs=False)
reconst_raw=bipolar_ref.copy()
  

EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=755505
    Range : 0 ... 755504 =      0.000 ...  3022.016 secs
Ready.
Added the following bipolar channels:
Reference


In [13]:
ica = ICA(n_components=15, max_iter='auto', random_state=97)
ica.fit(bipolar_ref)
eog_indices, eog_scores = ica.find_bads_eog(bipolar_ref, ch_name='Reference')
ica.exclude = eog_indices
ica.apply(reconst_raw)
    

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 19.0s.
Using EOG channel: Reference
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 sec)



[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 1 ICA component
    Projecting back using 64 PCA components


<RawBrainVision | sub-05_task-rsvp_eeg.eeg, 64 x 755505 (3022.0 s), ~369.0 MB, data loaded>

In [14]:
#Rimuovo il canale di reference perchè non mi serve più
final_raw=reconst_raw.drop_channels('Reference')